# 决策树与随机森林

<br>
<br>

- 参考：
   - 统计学习方法的代码实现

- 知识点
  1. 基于特征对实例进行分类的树形结构
  2. 试图找到最优的分割决策树（实际上是NP-C问题）
  3. 核心是计算每个属性的分类效果统计量（信息增益、信息增益比、基尼系数）
  4. 生成决策树时采用贪心思想递归划分子集
  5. 为解决过拟合问题，进行剪枝；往往从已生成的树上剪掉一些叶结点或叶结点以上的子树，并将其父结点或根结点作为新的叶结点，从而简化生成的决策树。

<br>
<br>

- 决策树家族
   - ID3 样本集合$D$对特征$A$的信息增益（ID3）
   
   $$g(D, A)=H(D)-H(D|A)$$

$$H(D)=-\sum_{k=1}^{K} \frac{\left|C_{k}\right|}{|D|} \log _{2} \frac{\left|C_{k}\right|}{|D|}$$

$$H(D | A)=\sum_{i=1}^{n} \frac{\left|D_{i}\right|}{|D|} H\left(D_{i}\right)$$
   
   - C4.5 样本集合$D$对特征$A$的信息增益比
   
   $$g_{R}(D, A)=\frac{g(D, A)}{H(D)}$$
   
   - CART 样本集合$D$的基尼指数
  
  $$\operatorname{Gini}(D)=1-\sum_{k=1}^{K}\left(\frac{\left|C_{k}\right|}{|D|}\right)^{2}$$

特征$A$条件下集合$D$的基尼指数：

 $$\operatorname{Gini}(D, A)=\frac{\left|D_{1}\right|}{|D|} \operatorname{Gini}\left(D_{1}\right)+\frac{\left|D_{2}\right|}{|D|} \operatorname{Gini}\left(D_{2}\right)$$
  
  - Random Forest
  
  
- 编程
   - 计算条件熵、信息熵、信息增益、信息增益比
      - 熵计算公式   $H(D)=-\sum_{k=1}^{K} \frac{\left|C_{k}\right|}{|D|} \log _{2} \frac{\left|C_{k}\right|}{|D|}$
      - 连续值如何计算熵
         - 参考：  [决策树（decision tree）(三)——连续值处理](https://blog.csdn.net/u012328159/article/details/79396893)
         - 将连续值观测量的观测结果排序，$a1,a2,a3,a4,...a_{n}$
         - 取中值$m1,m2,m3,m4,..,m_{n-1} = (a1+a2)/2,(a2+a3)/2,...,(a_{n-1}+a_{n})/2$
         - 按照每个中值$x>m_{i}$,$x<m_{i}$作为一个判断条件
         - 计算每个中值对应的信息增益，选出其中具有最大信息增益的点作为该连续属性的判断条件
     
  - 使用sklearn的决策树模型
 


In [1]:
# 计算信息增益
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
# 导入数据
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

def import_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data,columns = iris.feature_names)
    df['label'] = iris.target
    data = np.array(df)
    return data[:,:-1] , data[:,-1] , iris.feature_names




In [55]:
class DTree:
    def __init__(self):
        self.label = None
        self.data = None
        self.features = None
        self.entropy = None
        self.info_gain = None
        pass
    
    def train(self,features,label,names):
        self.features = features
        self.label = label
        label = label[:,np.newaxis]
        data = np.hstack((features,label))
        col_name = names.copy()
        
        col_name.append('label')
        self.data = pd.DataFrame(data,columns=col_name)
        
        

    def get_entropy(self,lis = None,attr = 'label'):
        # 接收一个列表('label'值)，返回这个列表的label的熵
        rec = False
        if lis is None:
            lis = list(self.data[attr])
            rec = True
            if self.entropy is not None:
                return self.entropy
        num = len(lis)
        cnt = dict(Counter(lis))
        
#         print(cnt)
        p = np.array(np.array(list(cnt.values()))/num)
        ent = np.sum(p*np.log2(p))
        ent = -ent
        if rec == True:
            self.entropy = ent
        return ent
        
    def get_cond_entropy(self,attr,val,lab_attr = 'label'):
        # H(D|A) = sum_each_a(p(a)*H(D|a))
        arr = self.data[attr]
        s1 = np.array(sorted(arr))
        t1 =  len(np.where(s1 < val))
        t = len(arr)
        p_a = [t1/t,1-t1/t]
        
#         print(np.where(val > s1))
        l1 = self.label[np.where(arr < val)]
        l2 = self.label[np.where(arr >= val)]
        H_cnd = [
            self.get_entropy(lis = l1),self.get_entropy(lis = l2)
        ]
        return p_a[0]*H_cnd[0] + p_a[1]*H_cnd[1]
    
    def get_info_gain(self,attr,lab_attr = 'label'):
        # 由于iris数据集属性的观测值是连续的，采用连续值熵计算公式
        arr = self.data[attr]
        s = sorted(arr,reverse=False)
        s.append(0)
        s1 = np.array(s[:-1])
        s2 = np.array(s[1:])     
        mid = (s1+s2)/2
        mid = mid.tolist()[:-1]
        mid_ig = {}
        for index,item in enumerate(mid):
            # 计算每个中值的信息增益
            # IG(D|A) = H(D) - H(D|A)
            mid_ig[index] = self.get_entropy() - self.get_cond_entropy(attr,item)
        ind = max(mid_ig,key = mid_ig.get)
        return mid[ind],mid_ig[ind]

In [57]:

# x,y,names = import_data()
# X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3)
dt = DTree()
# dt.train(X_train,y_train,names)
xx = np.array([1,2,2,3,4,5,6,33])
xx = xx[:,np.newaxis]
yy = np.array([0,1,0,0,1,1,1,1])
names = ['attr']
dt.train(xx,yy,names)
# print(y_train)

for i in names:
    print(i,'-->',dt.get_info_gain(i))

attr --> (3.5, 0.8530242373675735)


In [58]:
# sklearn 调用sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
# import graphviz

